**라이브러리, 데이터 가져오기**

book: 2023년 공앤박 책 데이터

kw: 2022년 키워드 추출했던 결과물

In [1]:
import pandas as pd
import json
import re

In [2]:
book = pd.read_csv("/content/drive/MyDrive/SCM_books/book_info.csv")

<ipython-input-2-5fe2a9e5a0b8>:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  book = pd.read_csv("/content/drive/MyDrive/SCM_books/book_info.csv")


In [ ]:
kw = pd.read_csv("/content/drive/MyDrive/산학 공부/데이터/final_kw_new.csv", encoding="cp949")

<ipython-input-3-80b81240098b>:1: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  kw = pd.read_csv("/content/drive/MyDrive/산학 공부/데이터/final_kw_new.csv", encoding="cp949")


In [3]:
book = book[['isbn', 'title', 'author', 'introduction', 'category']]

In [ ]:
kw = kw[['isbn', 'main_keywords']]

**책 데이터 카테고리 분리**

In [4]:
def split_category_1(x):
  x = str(x)
  x_list = x.split(" > ")
  return x_list[0]

def split_category_2(x):
  x = str(x)
  x_list = x.split(" > ")
  if len(x_list) > 1:
    return x_list[1]
  else:
    return '없음'

def split_category_3(x):
  x = str(x)
  x_list = x.split(" > ")
  if len(x_list) > 2:
    return x_list[2]
  else:
    return '없음'

def split_category_4(x):
  x = str(x)
  x_list = x.split(" > ")
  if len(x_list) > 3:
    return x_list[3]
  else:
    return '없음'

book['cat_1'] = book['category'].apply(split_category_1)
book['cat_2'] = book['category'].apply(split_category_2)
book['cat_3'] = book['category'].apply(split_category_3)
book['cat_4'] = book['category'].apply(split_category_4)

In [5]:
book.drop(['cat_1', 'cat_4'], axis = 1, inplace=True)

**책 데이터 전처리**

In [6]:
# 제목에서 괄호 안에 있는 거 지우기
def remove(x):
  x = str(x)
  x = re.sub(r'\([^)]*\)', '', x)
  x = re.sub(r'\[[^]]*\)', '', x)
  return x

book['title'] = book['title'].apply(remove)

In [7]:
# introduction 개수 확인(578704)

book[book['introduction'] != '없음'].count()

isbn            578704
title           578704
author          578704
introduction    578704
category        578704
cat_2           578704
cat_3           578704
dtype: int64

In [7]:
book = book[book['introduction'] != '없음']

In [8]:
# introduction 전처리
def cleaning(x):
  x = str(x)
  pattern = r'[\u4E00-\u9FFF]+'
  match = re.search(pattern, x)
  if match:
    return '없음'
  else:
    x = x.replace("<BR>", "")
    x = x.replace("<br>", "")
    x = x.replace("▶", "")
    x = x.replace("\\", "")
    return x

book['introduction'] = book['introduction'].apply(cleaning)
book = book[book['introduction'] != '없음']
book.reset_index(drop=True, inplace=True)

In [9]:
# 책에 해당 안 되는 정보 드랍하기

book = book[book['cat_2'] != '없음']
book = book[book['cat_2'] != 'ebook용품']
book = book[book['cat_2'] != '디지털용품']
book = book[book['cat_2'] != '애니 굿즈']
book = book[book['cat_2'] != '문구']
book = book[book['cat_3'] != '없음']
book = book[book['cat_3'] != '기타']
book = book[book['cat_3'] != '없음']
book = book[book['cat_3'] != '가방/파우치']
book = book[book['cat_3'] != '문구세트']

In [10]:
book.reset_index(drop=True, inplace=True)

In [11]:
cat_2 = book['cat_2']
cat_list = cat_2.unique().tolist()

cat_to_dict1 = []
for cat in cat_list:
  df = book[book['cat_2'] == cat]
  df_dic = df[['title', 'cat_2', 'author', 'introduction']].to_dict(orient="records")
  catdict = {}
  catdict['category'] = cat
  catdict['book_list'] = df_dic
  cat_to_dict1.append(catdict)

In [12]:
cat_3 = book['cat_3']
cat_list = cat_3.unique().tolist()

cat_to_dict2 = []
for cat in cat_list:
  df = book[book['cat_3'] == cat]
  df_dic = df[['title', 'cat_3', 'author', 'introduction']].to_dict(orient="records")
  catdict = {}
  catdict['category'] = cat
  catdict['book_list'] = df_dic
  cat_to_dict2.append(catdict)

In [13]:
import random

for dictionary in cat_to_dict1:
  random.shuffle(dictionary['book_list'])
  each_list = dictionary['book_list']

  if len(each_list) > 50:
    dictionary['book_list'] = each_list[:60]

In [14]:
for dictionary in cat_to_dict2:
  random.shuffle(dictionary['book_list'])
  each_list = dictionary['book_list']

  if len(each_list) > 50:
    dictionary['book_list'] = each_list[:60]

In [ ]:
#@title

reduced_book_list = []
for dictionary in cat_to_dict:
  book_list = dictionary['book_list']
  reduced_book_list.append(book_list)
#@title
df = pd.DataFrame(columns=['isbn', 'title', 'cat_3', 'author', 'introduction'])

for cat in reduced_book_list:
  cat_df = pd.DataFrame(cat)
  df = pd.concat([df, cat_df], ignore_index=True)
# #@title
# final_df = pd.merge(df, kw, on='isbn', how='left')
# #@title
# final_df

In [15]:
def has_coda(word):
    return (ord(word[-1]) - 44032) % 28 == 0

In [22]:
prompt_list = []
ppo_list = []
for dictionary in cat_to_dict1:
  each_list = dictionary['book_list']
  for book in each_list:
    author = book['author']
    title = book['title']
    if author != '없음':
      if has_coda(author) == False: # 받침 있는 경우
        prompt = author + "이 쓴 책 추천해 줘"
        completion = author + "이 쓴 책으로는 "
        if has_coda(title) == False: # 책 제목에 받침
          completion += title + "을 추천합니다. "
        elif has_coda(title) == True:
          completion += title + "를 추천합니다. "
        completion += "이 책은 " + book['cat_2'] + "에 관한 책입니다. "
        if book['introduction'] != "없음":
          if has_coda(title) == False:
            completion += title + "은 " + book['introduction'][:150] + " 이러한 이유로 이 책을 추천합니다. "
          elif has_coda(title) == True:
            completion += title + "는 " + book['introduction'][:150] + " 이러한 이유로 이 책을 추천합니다. "
        completion += "즐거운 독서 되세요!"
        prompt_dict = {}
        prompt_dict['prompt'] = prompt
        prompt_dict['completion'] = completion
        prompt_list.append(prompt_dict)

In [23]:
for dictionary in cat_to_dict2:
  each_list = dictionary['book_list']
  i = 0
  prompt = ""
  completion = ""
  for book in each_list:
    i = i + 1
    if book['cat_3'] != "없음":
      prompt = book['cat_3'] + "에 대한 책 추천해 줘"
      completion += str(i) + ". " + book['title'] + ". "
      author = book['author']
      title = book['title']
      intro = book['introduction']
      if author != "없음":
        if has_coda(title) == False:
          completion += title + "은 "
        elif has_coda(title) == True:
          completion += title + "는 "
        if has_coda(author) == False:
          completion += author + "이 쓴 책입니다. "
        elif has_coda(author) == True:
          completion += author + "가 쓴 책입니다. "

      if intro != "없음":
        if has_coda(title) == False:
          completion += title + "은 " + book['introduction'][:150] + " "
        elif has_coda(title) == True:
          completion += title + "는 " + book['introduction'][:150] + " "
      if i == 3:
        completion += "이러한 책들을 추천합니다. 즐거운 독서 되세요!"
        break
  prompt_dict = {}
  prompt_dict['prompt'] = prompt
  prompt_dict['completion'] = completion
  prompt_list.append(prompt_dict)

In [ ]:
prompt_list = []
ppo_list = []
for dictionary in cat_to_dict1:
  each_list = dictionary['book_list']
  for book in each_list:
    if book['author'] != '없음':
      prompt = book['author'] + "가 쓴 책 추천해 줘"
      completion = book['author'] + "가 쓴 책으로는 " + book['title'] + "을 추천합니다. "
      completion = "이 책은 " + book['cat_2'] + "에 관한 책입니다. "
      if book['introduction'] != '없음':
        completion += book['title'] + "은 " + book['introduction'] + " 이러한 이유로 이 책을 추천합니다. "
      completion += "즐거운 독서 되세요!"
      prompt_dict = {}
      prompt_dict['prompt'] = prompt
      prompt_dict['completion'] = completion
      prompt_list.append(prompt_dict)
      ppo_list.append(prompt)

for dictionary in cat_to_dict2:
  each_list = dictionary['book_list']
  i = 0
  prompt = ""
  completion = ""
  for book in each_list:
    i = i + 1
    if book['cat_3'] != '없음':
      prompt = book['cat_3'] + "에 대한 책 추천해 줘"
      completion += str(i) + ". " + book['title'] + ". "
      if book['author'] != '없음':
        completion += book['title'] + "은 " + book['author'] + "가 쓴 책입니다. "
      if book['introduction'] != "없음":
        completion += book['title'] + "은 " + book['introduction'][:150] + " "
    if i == 3:
      completion += "이러한 책들을 추천합니다. 즐거운 독서 되세요!"
      break
  prompt_dict = {}
  prompt_dict['prompt'] = prompt
  prompt_dict['completion'] = completion
  prompt_list.append(prompt_dict)
  ppo_list.append(prompt)

In [ ]:
ppo_list

In [ ]:
with open('ppo_prompt.txt', 'w') as file:
    # Step 2: Convert the list to a string representation
    list_str = str(ppo_list)

    # Step 3: Write the string representation to the file
    file.write(list_str)

In [ ]:
prompt_list[-1]

{'prompt': '자동차에 대한 책 추천해 줘',
 'completion': '1. 더클래스. 더클래스은 더클래스효성 편집부가 쓴 책입니다. 더클래스은 더클래스 효성에서 발간하는 계간지 는 벤츠 고객의 품격과 명성에 걸맞은 Premium Lifestyle과 벤츠 오너라면 누구에게나 필요한 올바르고 합리적인 자동차 메인터넌스 정보, 꼭 알아야 할 자동차 및 생활, 문화, 여행 정보 등 벤츠 오너들의 생생한 경험과 이야기 2. 라이드 매거진. 라이드 매거진은 라이더컴퍼니 편집부가 쓴 책입니다. 라이드 매거진은 라이드컴퍼니에서 발행한 월간 라이드매거진은 국내 최고의 전통과 권위를 자랑하는 모터사이크 전문 잡지이다. 3. 오토모티브 일렉트로닉 매거진 AEM. 오토모티브 일렉트로닉 매거진 AEM은 스마트앤컴퍼니 편집부가 쓴 책입니다. 오토모티브 일렉트로닉 매거진 AEM은 automotive electronics magazine 이러한 책들을 추천합니다. 즐거운 독서 되세요!'}

In [24]:
len(prompt_list)

8730

In [25]:
import re

for dictionary in prompt_list:
  completion = dictionary['completion']
  completion = completion.replace("그리고 없음", "")
  completion = completion.replace("<br>", "")
  completion = completion.replace("<BR>", "")
  completion = completion.replace('\\', '')
  completion = completion.replace("▶", "")
  completion = completion.replace("···", "")
  completion = completion.replace("[양장본]", "")
  completion = re.sub(r"\s+", " ", completion)
  dictionary['completion'] = completion

In [27]:
prompt_list[0]

{'prompt': '허정이 쓴 책 추천해 줘',
 'completion': "허정이 쓴 책으로는 아함경: 재가자 편을 추천합니다. 이 책은 불교에 관한 책입니다. 아함경: 재가자 편은 『아함경: 재가자 편』은 우리가 알아야 할 부처님 말씀 '잡아함' 중에서 주로 재가신도를 위한 경을 주제별로 가려엮은 한글 아함경이다. 아함경은 부처님의 인간미에 가깝고 대ㆍ소승의 구별도 보이지 않는 불교의 근본이 되는 가르침을 기록한 경전이다. 이러한 이유로 이 책을 추천합니다. 즐거운 독서 되세요!"}

In [28]:
with open("sft_data.json", "w", encoding="utf8") as json_file:
    json.dump(prompt_list, json_file, ensure_ascii=False)

**RM 데이터 형식 새로 만들기**

In [ ]:
with open('/content/drive/MyDrive/SCM_books/rm_dataset.json', "r", encoding='utf-8') as json_file:
    data = json.load(json_file)

In [ ]:
data[0]

{'prompt': '초등학생이 읽을 만한 책 추천해 줘',
 'completion_0': '\n\n1. 바나나 같은 나영어 시 by 백혜선  \n2. 별빛 소녀 엘레나 by 강이채  \n3. 좋은 은사에 샤워하라! by 이하늬  \n4. 여우 라인 by 배시  \n5. 별빛 나무에 들어간 라이트 by 김해린  \n6. 개에게서 배운 책 by 이마리  \n7. 화학 소녀의 모든 것 by 윤석희',
 'completion_1': "1. '으라차차 와이키키' 시리즈 - 김수현 작가\n2. '빨간 머리 앤' 시리즈 - L.M. 몽고메리 작가\n3. '공룡 100%' 시리즈 - 피터 마일즈 작가\n4. '누구나 인생에 실패할 수밖에 없다' - 홍성욱 작가\n5. '사랑은 미친짓이다' - 김지혜 작가\n6. '나의 머니 북' 시리즈 - 강주현 작가\n7. '초등 영어 동화' 시리즈 - 트랜스미디어 작가\n8. '우리 아이의 손맛 레시피' - 송영주 작가\n9. '태극기가 휘날리며' 시리즈 - 이준혁 작가\n10. '그래서 하루는 달빛이 비쳤다' - 권주현 작가",
 'completion_2': '요\n\n초등학생이 읽을 만한 책에서 가장 잘 읽을 수 있는 책이 있습니다. 그리고 이 책을 기반으로 학생에게 하는 경험이 더 높은 책임을 갖고 있는 책입니다.',
 'ranking': ['2', '1', '3']}

In [ ]:
with open('/content/drive/MyDrive/SCM_books/SFT_book_recommendation_prompt_revised_no_titleauthor.json', "r", encoding='utf-8') as json_file:
    data = json.load(json_file)

In [ ]:
data[0]

{'prompt': 'book_recommendation: {input에 주어진 책 정보와 사용자 질의를 바탕으로 그 책을 추천한 사유를 생성해줘}',
 'input': 'user_query: {초등학생들이 재미있게 읽을 수 있는 조류의 생태에 대해 다룬 이야기를 읽을 수 있는 책을 추천해주세요.}, book: {title: [조류 세계에서 살아남기 1-2권 세트], author: [곰돌이 co.], introduction: [조류 세계에서 살아남기 1~2권 세트는 새들의 섬으로 간 지오가 하늘을 나는 탐조복을 입고 새들과 벌이는 모험 이야기입니다. 흥미진진한 모험 속에 참매나 흰꼬리수리 같은 맹금류부터 주위에서 보기 드문 키위나 바우어 새에 이르기까지 다양한 조류들이 등장합니다. 주인공들이 온갖 새들과 맞서는 과정을 통해 조류의 특징과 생태에 대해 배워봅니다. 한층 더 업그레이드된 재미와 정보, 생생한 이야기를 친근하고 역동적인 그림을 통해 만나 보세요!]}',
 'completion': '- 이 책은 초등학생들이 재미있게 읽을 수 있는 조류의 생태에 대해 다룬 이야기로 구성되어 있습니다. 주인공들이 다양한 조류들과 모험을 벌이는 과정을 통해 새들의 특징과 생태를 쉽게 배울 수 있습니다. 이 책은 생생한 그림과 역동적인 이야기를 통해 초등학생들이 흥미를 느끼며 독서를 할 수 있도록 제작되었습니다.'}